In [62]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import re
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

In [63]:
sys.path.append('../')
import read_player_stats
import training_data
import game_by_game

In [64]:
positions = ['qb', 'rb', 'wr', 'te']
df_list = [pd.read_csv('../game_stats/'+ pos +'_game_stats.csv') for pos in positions]
df_list = [df.drop(['Unnamed: 0'], axis = 1) for df in df_list]

In [65]:
df_list[0].head()

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week
0,AJFeeley,MIA,1,21,31,168,1,1,0,0,0,12.4,12.4,2004,1
1,AJFeeley,MIA,1,21,39,218,1,2,1,3,0,15.2,15.2,2004,2
2,AJFeeley,MIA,1,13,27,137,0,2,4,1,0,7.0,7.0,2004,3
3,AJFeeley,MIA,1,1,2,6,0,0,0,0,0,0.3,0.3,2004,5
4,AJFeeley,MIA,1,6,15,129,0,1,0,0,0,6.5,6.5,2004,9


In [66]:
tdata_list = [game_by_game.gbg_train_stats(df) for df in df_list]

../game_by_game.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  stats_cols = temp_df.drop(['Name', 'Team', 'Games', 'Season', 'Week','FFPPG'], axis=1).columns


In [67]:
tdata_list[0].head(10)

,Label,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,WeekID_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,WeekID_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,WeekID_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,WeekID_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,WeekID_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,WeekID_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,WeekID_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,WeekID_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,WeekID_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,WeekID_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,WeekID_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,WeekID_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,WeekID_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,WeekID_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,WeekID_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,WeekID_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,WeekID_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,WeekID_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,WeekID_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19,WeekID_19,Name
0,-1.0,12,22,156,0,1,0,0,0,7.8,131,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,AJFeeley
1,7.8,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,1,2,6,0,0,0,0,0,0.3,4,AJFeeley
2,0.0,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,1,2,6,0,0,0,0,0,0.3,4,13,27,137,0,2,4,1,0,7.0,2,AJFeeley
3,13.3,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1

In [105]:
orig = {'Season': 2004, 'Week': 1}
curr_week = 17*(2015 - orig['Season']) + (1 - orig['Week'])
test_samples = [df[(df.Label == -1) & (df.WeekID_0 > curr_week-5)] for df in tdata_list]
train_samples = [df[df.Label != -1] for df in tdata_list]

In [106]:
week_labels = ['WeekID_%d' % i for i in range(20)]

In [107]:
test_samples = [df.drop(week_labels, axis=1) for df in test_samples]
train_samples = [df.drop(week_labels + ['Name'], axis=1) for df in train_samples]

In [108]:
y_train = [df['Label'] for df in train_samples]
X_train = [np.array(df.drop(['Label'], axis=1)) for df in train_samples]

In [109]:
rfr = [RandomForestRegressor(n_estimators=200) for i in range(4)]

In [110]:
for i in range(4):
    rfr[i].fit(X_train[i], y_train[i])

In [111]:
test_samples[0].drop(['Label', 'Name'], axis=1)

,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19
0,17,22,226,2,0,2,14,1,26.7,31,40,318,1,0,0,0,0,19.9,17,42,185,0,2,3,27,0,12.0,24,36,327,3,0,5,28,0,31.2,24,38,368,2,0,5,22,0,28.6,19,29,209,2,0,6,34,0,21.9,22,36,341,3,0,3,32,0,32.3,18,27,315,6,0,0,0,0,39.8,28,39,418,1,2,2,21,1,33.0,19,22,255,3,0,3,21,0,26.9,24,42,264,3,0,7,34,0,28.6,12,17,156,3,0,0,0,0,19.8,22,28,302,4,0,1,8,0,31.9,16,27,162,1,0,0,0,0,12.1,25,42,346,3,0,6,28,0,32.1,23,33,189,1,1,0,0,0,13.5,25,39,318,2,2,1,5,0,24.4,1,2,27,0,0,1,9,0,2.3,24,29,285,2,0,6,31,0,25.4,25,36,260,3,0,4,12,0,26.2
0,31,45,311,0,0,2,14,0,17.0,18,30,297,2,0,3,17,0,24.6,26,39,293,1,1,4,26,0,21.3,15,23,153,2,1,5,6,0,16.3,20,36,234,2,0,2,5,0,20.2,11,16,108,0,0,2,-1,0,5.3,17,29,177,0,0,4,25,1,17.4,21,31,199,2,0,0,0,0,18.0,24,28,226,0,0,4,24,0,13.7,19,28,221,1,0,6,29,0,18.0,16,30,158,1,1,1,6,0,12.5,20,26,248,3,0,4,8,0,25.2,19,25,186,3,0,2,17,0,23.0,26,42,255,0,0,5,42,0,17.0,19,35,202,1,3,6,36,0,17.7,16,29,153,0,2,6,47,0,12.4,17,20,287,5,0,4,17,0,36.1,14,20,137,2,0,2,7,0,15.6,26,42,293,2,1,4,46,0,27.3,26,38,294,3,1,2,-3,0,26.4
0,10,16,160,2,0,1,5,0,16.5,15,22,109,0,2,0,0,0,5.5,18,34,187,2,1,8,18,0,19.2,24,53,294,2,2,5,37,1,32.4,19,27,370,5,1,4,20,0,40.5,21,32,253,1,0,8,49,0,21.6,23,39,303,2,1,3,15,0,24.7,25,46,354,4,0,2,5,0,34.2,26,45,400,3,2,3,26,0,34.6,27,42,344,2,0,4,5,0,25.7,25,44,370,3,1,7,18,0,32.3,32,49,312,1,2,6,12,1,26.8,29,41,393,4,1,1,-1,0,35.6,31,39,370,4,0,3,15,0,36.0,20,34,172,3,1,4,20,0,22.6,35,53,370,2,2,5,19,1,34.4,26,37,282,1,0,2,7,0,18.8,26,37,241,1,0,6,5,0,16.6,19,32,180,2,1,5,29,0,19.9,29,46,326,4,0,2,32,0,35.5
0,27,38,244,2,2,3,8,0,21.0,17,26,146,2,1,6,25,0,17.8,14,24,117,0,1,2,3,0,6.2,21,29,302,2,0,4,22,1,31.3,19,27,176,1,3,3,7,1,19.5,24,35,233,1,1,7,6,0,16.3,16,22,220,3,0,5,12,0,24.2,10,33,86,0,3,3,8,0,5.1,19,31,233,2,2,2,11,0,20.8,21,28,266,0,1,6,14,2,26.7,18,38,126,0,0,1,0,0,6.3,34,44,323,2,2,4,25,0,26.7,15,24,204,2,0,2,16,0,19.8,15,23,169,0,1,3,3,0,8.8,15,23,252,1,0,3,6,0,17.2,25,38,301,1,0,6,3,0,19.4,21,36,281,2,4,6,23,1,30.4,27,38,366,4,0,1,10,0,35.3,25,44,230,2,0,4,20,0,21.5,24,35,275,3,0,5,11,1,32.9
0,24,38,317,2,1,4,3,0,24.2,18,25,220,1,0,4,-4,0,14.6,27,35,360,0,0,2,-2,0,17.8,25,39,350,3,0,1,-1,0,29.4,32,58,435,2,2,1,8,0,30.6,21,32,207,1,1,3,2,0,14.6,30,43,343,1,2,1,0,0,21.2,25,37,340,6,0,2,1,0,41.1,40,49,522,6,0,1,

In [112]:
preds = [rfr[i].predict(np.array(test_samples[i].drop(['Label', 'Name'], axis=1))) for i in range(4)]

In [120]:
proj_list = []
for i in range(4):
    test_samples[i]['Projection'] = preds[i]
    pro_df = test_samples[i][['Name', 'Projection']]
    proj_list.append(pro_df.sort(columns=['Projection'], ascending=False))

In [122]:
proj_list[0]

,Name,Projection
0,AndrewLuck,23.3840
0,DrewBrees,22.9435
0,BenRoethlisberger,22.8450
0,AaronRodgers,22.7855
0,MattRyan,22.6905
0,JoeFlacco,22.1075
0,PeytonManning,22.0170
0,JayCutler,21.5250
0,EliManning,21.0125
0,TomBrady,20.9845


In [123]:
proj_list[1]

,Name,Projection
0,LeVeonBell,20.4185
0,DeMarcoMurray,18.6660
0,MattForte,18.5335
0,MarshawnLynch,17.7310
0,ArianFoster,15.7765
0,EddieLacy,15.4750
0,LeSeanMcCoy,15.1045
0,FredJackson,14.3685
0,JamaalCharles,14.1460
0,JoiqueBell,13.8355


In [124]:
proj_list[2]

,Name,Projection
0,JulioJones,19.7085
0,JordyNelson,19.3850
0,JoshGordon,18.8535
0,AntonioBrown,18.8190
0,RandallCobb,18.4770
0,DezBryant,17.9530
0,CalvinJohnson,17.7605
0,AJGreen,17.0045
0,DemaryiusThomas,16.3430
0,JulianEdelman,16.2050


In [125]:
proj_list[3]

,Name,Projection
0,RobGronkowski,14.9770
0,JimmyGraham,13.4550
0,MartellusBennett,12.7850
0,GregOlsen,12.5160
0,ZachErtz,11.7720
0,DelanieWalker,11.4380
0,AntonioGates,11.4255
0,JasonWitten,10.6905
0,OwenDaniels,10.5555
0,CharlesClay,10.1190
